# Forecasting Stickers Sale

In [46]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import pandas as pd

test = pd.read_csv('/content/drive/MyDrive/Kaggle/playground-series-s5e1/test.csv')
train = pd.read_csv('/content/drive/MyDrive/Kaggle/playground-series-s5e1/train.csv')
print(test.head())
print(train.head())


       id        date country              store             product
0  230130  2017-01-01  Canada  Discount Stickers   Holographic Goose
1  230131  2017-01-01  Canada  Discount Stickers              Kaggle
2  230132  2017-01-01  Canada  Discount Stickers        Kaggle Tiers
3  230133  2017-01-01  Canada  Discount Stickers            Kerneler
4  230134  2017-01-01  Canada  Discount Stickers  Kerneler Dark Mode
   id        date country              store             product  num_sold
0   0  2010-01-01  Canada  Discount Stickers   Holographic Goose       NaN
1   1  2010-01-01  Canada  Discount Stickers              Kaggle     973.0
2   2  2010-01-01  Canada  Discount Stickers        Kaggle Tiers     906.0
3   3  2010-01-01  Canada  Discount Stickers            Kerneler     423.0
4   4  2010-01-01  Canada  Discount Stickers  Kerneler Dark Mode     491.0


In [48]:
# Display information about the test dataset
print("Test Data Description:")
print(test.info())
print(test.describe())
print("\n")

# Display information about the training dataset
print("Train Data Description:")
print(train.info())
print(train.describe())

Test Data Description:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98550 entries, 0 to 98549
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       98550 non-null  int64 
 1   date     98550 non-null  object
 2   country  98550 non-null  object
 3   store    98550 non-null  object
 4   product  98550 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
                  id
count   98550.000000
mean   279404.500000
std     28449.078852
min    230130.000000
25%    254767.250000
50%    279404.500000
75%    304041.750000
max    328679.000000


Train Data Description:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null

In [49]:
# Fill NaN values in 'num_sold' with the mean
#train['num_sold'] = train['num_sold'].fillna(train['num_sold'].median())



In [50]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error


# Preprocessing: Convert date to datetime and extract features
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

# Extract year, month, day, and day_of_week from date
for df in [train, test]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek

# Encode categorical features
label_encoder = LabelEncoder()
for col in ['country', 'store', 'product']:
    train[col] = label_encoder.fit_transform(train[col])
    test[col] = label_encoder.transform(test[col])

# Handle missing values in 'num_sold' using predictive modeling
train_known = train[train['num_sold'].notna()]
train_missing = train[train['num_sold'].isna()]

if not train_missing.empty:
    # Train a model to fill missing 'num_sold'
    X_known = train_known.drop(columns=['num_sold', 'date', 'id'])
    y_known = train_known['num_sold']
    X_missing = train_missing.drop(columns=['num_sold', 'date', 'id'])

    imputer_model = RandomForestRegressor(random_state=42)
    imputer_model.fit(X_known, y_known)
    train.loc[train['num_sold'].isna(), 'num_sold'] = imputer_model.predict(X_missing)

# Define features and target for main model
X_train = train.drop(columns=['num_sold', 'date', 'id'])
y_train = train['num_sold']
X_test = test.drop(columns=['date', 'id'])

# Split training data for validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the main model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_split, y_train_split)

# Evaluate the model on validation data
y_val_pred = model.predict(X_val)
print("Validation MAE:", mean_absolute_error(y_val, y_val_pred))

# Predict on test data
test['num_sold'] = model.predict(X_test)

# Save the predictions to a CSV
test[['id', 'num_sold']].to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


Validation MAE: 37.75357523573633
Predictions saved to 'predictions.csv'.


In [60]:
# prompt: download saved predictions.csv file

from google.colab import files
files.download('predictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
# prompt: Remove rows of nan values in num_sold

# Remove rows with NaN values in the 'num_sold' column
# train = train.dropna(subset=['num_sold'])

In [52]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import LabelEncoder

# # Data preprocessing
# train['date'] = pd.to_datetime(train['date'])
# test['date'] = pd.to_datetime(test['date'])

# # Feature engineering: Extract features from the date
# train['year'] = train['date'].dt.year
# train['month'] = train['date'].dt.month
# train['day'] = train['date'].dt.day
# train['day_of_week'] = train['date'].dt.dayofweek

# test['year'] = test['date'].dt.year
# test['month'] = test['date'].dt.month
# test['day'] = test['date'].dt.day
# test['day_of_week'] = test['date'].dt.dayofweek

# # Encoding categorical features
# label_encoder = LabelEncoder()
# train['country'] = label_encoder.fit_transform(train['country'])
# test['country'] = label_encoder.transform(test['country'])
# train['store'] = label_encoder.fit_transform(train['store'])
# test['store'] = label_encoder.transform(test['store'])
# train['product'] = label_encoder.fit_transform(train['product'])
# test['product'] = label_encoder.transform(test['product'])

# # Define the features and target
# X_train = train.drop(columns=['num_sold', 'date', 'id'])
# y_train = train['num_sold']
# X_test = test.drop(columns=['date', 'id'])

# # Train the model
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # Predict on test data
# predictions = model.predict(X_test)

# # Optionally, evaluate on the training data if a validation set is available
# y_train_pred = model.predict(X_train)
# print("Training MAE:", mean_absolute_error(y_train, y_train_pred))

# # Save predictions to the test dataset
# test['pred_num_sold'] = predictions

In [53]:
test.head()

,id,date,country,store,product,year,month,day,day_of_week,num_sold
0,230130,2017-01-01,0,0,0,2017,1,1,6,137.9549
1,230131,2017-01-01,0,0,1,2017,1,1,6,711.8800
2,230132,2017-01-01,0,0,2,2017,1,1,6,676.4800
3,230133,2017-01-01,0,0,3,2017,1,1,6,357.6900
4,230134,2017-01-01,0,0,4,2017,1,1,6,405.2400


In [54]:
# prompt: create a df with column values from test id and pred_num_sold

# Create a DataFrame with 'test_id' and 'pred_num_sold'
df = pd.DataFrame({'test_id': test['id'], 'pred_num_sold': predictions})

# Display the first few rows of the submission DataFrame
print(df.head())

   test_id  pred_num_sold
0   230130         143.83
1   230131         761.96
2   230132         669.79
3   230133         361.49
4   230134         405.70


In [55]:
# prompt: rename column name test_id to id & pred_num_sold to num_sold and need column num_sold as round value

# Rename columns
df = df.rename(columns={'test_id': 'id', 'pred_num_sold': 'num_sold'})

# Round the 'num_sold' column
df['num_sold'] = df['num_sold'].round()
df['num_sold'] = df['num_sold'].astype(int)

# Display the first few rows of the updated DataFrame
print(df.head())

       id  num_sold
0  230130       144
1  230131       762
2  230132       670
3  230133       361
4  230134       406


In [56]:
df.shape


(98550, 2)

In [59]:
# prompt: Download df file in csv format

from google.colab import files
df.to_csv('FSSP.csv', index=False)
files.download('FSSP.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>